This Code is to drop all unused features from Forecast Data

"SolarIntensity = F(Day, Temperature, DewPoint, WindSpeed, SkyCover, Precipitation, Humidity)"

Then, to replace missing values/ drop unused rows and rename days

We only have forecast at t-6h and not t-3h

In [29]:
# Step 1: Import necessary libraries
import pandas as pd

# Step 2: Load the CSV file
file_path = "Raw Extracted Datasets/Forecast NWS.csv"
df = pd.read_csv(file_path)

df.head()

,runtime,ftime,model,n_x,tmp,dpt,cld,wdr,wsp,p06,...,snw,cig,vis,obv,poz,pos,typ,station,t06,t12
0,2009-12-31 00:00:00,2009-12-31 06:00:00,GFS,NaN,21,15,OV,160,1,NaN,...,NaN,7,7,N,5.0,95.0,S,KBAF,NaN,NaN
1,2009-12-31 00:00:00,2009-12-31 09:00:00,GFS,NaN,20,14,OV,200,1,NaN,...,NaN,7,7,N,8.0,92.0,S,KBAF,NaN,NaN
2,2009-12-31 00:00:00,2009-12-31 12:00:00,GFS,NaN,19,13,OV,180,1,1.0,...,NaN,7,7,N,9.0,91.0,S,KBAF,0/0,NaN
3,2009-12-31 00:00:00,2009-12-31 15:00:00,GFS,NaN,23,16,OV,0,0,NaN,...,NaN,6,7,N,11.0,86.0,S,KBAF,NaN,NaN
4,2009-12-31 00:00:00,2009-12-31 18:00:00,GFS,NaN,26,20,OV,210,1,51.0,...,NaN,5,5,BR,10.0,75.0,S,KBAF,1/0,NaN


In [30]:
# Step 3: Convert runtime and ftime to datetime
df["runtime"] = pd.to_datetime(df["runtime"])
df["ftime"] = pd.to_datetime(df["ftime"])

# Step 4: Create separate columns for date and time for runtime and ftime
df["RuntimeDate"] = df["runtime"].dt.date
df["RuntimeTime"] = df["runtime"].dt.time
df["ForecastDate"] = df["ftime"].dt.date
df["ForecastTime"] = df["ftime"].dt.time

# Step 5: Select and rename relevant columns (excluding pos)
df_cleaned = df[[
    "RuntimeDate", "RuntimeTime",
    "ForecastDate", "ForecastTime",
    "tmp", "dpt", "wsp", "cld", "p06"
]].copy()

df_cleaned.columns = [
    "RuntimeDate", "RuntimeTime",
    "ForecastDate", "ForecastTime",
    "Temperature", "DewPoint", "WindSpeed", "SkyCover", "Precipitation"
]

# Display the cleaned DataFrame
df_cleaned.head()


,RuntimeDate,RuntimeTime,ForecastDate,ForecastTime,Temperature,DewPoint,WindSpeed,SkyCover,Precipitation
0,2009-12-31,00:00:00,2009-12-31,06:00:00,21,15,1,OV,NaN
1,2009-12-31,00:00:00,2009-12-31,09:00:00,20,14,1,OV,NaN
2,2009-12-31,00:00:00,2009-12-31,12:00:00,19,13,1,OV,1.0
3,2009-12-31,00:00:00,2009-12-31,15:00:00,23,16,0,OV,NaN
4,2009-12-31,00:00:00,2009-12-31,18:00:00,26,20,1,OV,51.0


In [32]:
from datetime import datetime, time
# Step 6: Print null and non-null counts for non-runtime and forecast columns
print("Null value counts per column:\n")
print(df_cleaned.drop(columns=["RuntimeDate", "RuntimeTime", "ForecastDate", "ForecastTime"]).isnull().sum())
print("\nNon-null value counts per column:\n")
print(df_cleaned.drop(columns=["RuntimeDate", "RuntimeTime", "ForecastDate", "ForecastTime"]).notnull().sum())

# Step 7: Fill null values using mean for numeric columns and mode for categorical columns
# Numeric columns
numeric_cols = ["Temperature", "DewPoint", "WindSpeed", "Precipitation"]
for col in numeric_cols:
    mean_value = df_cleaned[col].mean()
    df_cleaned[col] = df_cleaned[col].fillna(mean_value)  # Avoid inplace=True

# Categorical columns
categorical_cols = ["SkyCover"]
for col in categorical_cols:
    mode_value = df_cleaned[col].mode()[0]
    df_cleaned[col] = df_cleaned[col].fillna(mode_value)  # Avoid inplace=True

# Display the cleaned DataFrame
df_cleaned


Null value counts per column:

Temperature      0
DewPoint         0
WindSpeed        0
SkyCover         0
Precipitation    0
dtype: int64

Non-null value counts per column:

Temperature      25704
DewPoint         25704
WindSpeed        25704
SkyCover         25704
Precipitation    25704
dtype: int64


,RuntimeDate,RuntimeTime,ForecastDate,ForecastTime,Temperature,DewPoint,WindSpeed,SkyCover,Precipitation
0,2009-12-31,00:00:00,2009-12-31,06:00:00,21,15,1,OV,16.766563
1,2009-12-31,00:00:00,2009-12-31,09:00:00,20,14,1,OV,16.766563
2,2009-12-31,00:00:00,2009-12-31,12:00:00,19,13,1,OV,1.000000
3,2009-12-31,00:00:00,2009-12-31,15:00:00,23,16,0,OV,16.766563
4,2009-12-31,00:00:00,2009-12-31,18:00:00,26,20,1,OV,51.000000
...,...,...,...,...,...,...,...,...,...
25699,2010-11-01,00:00:00,2010-11-03,06:00:00,25,19,1,CL,3.000000
25700,2010-11-01,00:00:00,2010-11-03,09:00:00,24,19,1,CL,16.766563
25701,2010-11-01,00:00:00,2010-11-03,12:00:00,25,20,1,CL,3.000000
25702,2010-11-01,00:00:00,2010-11-03,18:00:00,47,22,5,CL,3.000000


In [39]:
from datetime import datetime, time

# Step 1: Filter RuntimeDate between Jan 1 and Oct 31, 2010
start_date = datetime(2010, 1, 1).date()
end_date = datetime(2010, 10, 31).date()
mask_date = (df_cleaned["RuntimeDate"] >= start_date) & (df_cleaned["RuntimeDate"] <= end_date)

# Step 2: Filter for RuntimeTime == 6:00:00 (6 AM)
mask_runtime_time = df_cleaned["RuntimeTime"] == time(6, 0)

# Step 3: Filter for ForecastTime == 12:00:00 (12 AM)
mask_forecast_time = df_cleaned["ForecastTime"] == time(12, 0)

# Combine all filters
df_filtered = df_cleaned[mask_date & mask_runtime_time & mask_forecast_time].copy()

# Step 4: Drop rows where runtime date is not equal to forecast date
df_filtered = df_filtered[df_filtered["RuntimeDate"] == df_filtered["ForecastDate"]].copy()

# Drop unnecessary columns
df_final = df_filtered.drop(columns=["RuntimeTime", "ForecastDate", "ForecastTime"])

# Rename RuntimeDate to Day
df_final = df_final.rename(columns={"RuntimeDate": "Day"})

# Reset index
df_final = df_final.reset_index(drop=True)

# Show the cleaned DataFrame
df_final.head()


# Display the filtered data
df_final


,Day,Temperature,DewPoint,WindSpeed,SkyCover,Precipitation
0,2010-01-01,28,25,1,OV,16.766563
1,2010-01-02,30,23,10,OV,16.766563
2,2010-01-03,17,4,15,OV,16.766563
3,2010-01-04,21,11,9,SC,16.766563
4,2010-01-05,20,10,7,OV,16.766563
...,...,...,...,...,...,...
302,2010-10-27,68,65,6,OV,16.766563
303,2010-10-28,51,49,1,OV,16.766563
304,2010-10-29,42,37,2,BK,16.766563
305,2010-10-30,34,29,3,SC,16.766563


In [40]:
# Step 10: Print all unique values in the 'SkyCover' column
unique_skycover_values = df_cleaned["SkyCover"].unique()
print("Unique values in SkyCover:", unique_skycover_values)

# Step 8: Define a mapping dictionary for SkyCover based on the unique values
skycover_mapping = {
    "OV": 100,  # Overcast
    "BK": 80,   # Broken Clouds
    "FW": 25,   # Few Clouds/Fair Weather
    "CL": 0,    # Clear
    "SC": 40    # Scattered Clouds
}

# Step 9: Apply the mapping to the 'SkyCover' column
df_final["SkyCover"] = df_final["SkyCover"].map(skycover_mapping)

# Display the updated DataFrame with the mapped SkyCover percentages
df_final


Unique values in SkyCover: ['OV' 'BK' 'FW' 'CL' 'SC']


,Day,Temperature,DewPoint,WindSpeed,SkyCover,Precipitation
0,2010-01-01,28,25,1,100,16.766563
1,2010-01-02,30,23,10,100,16.766563
2,2010-01-03,17,4,15,100,16.766563
3,2010-01-04,21,11,9,40,16.766563
4,2010-01-05,20,10,7,100,16.766563
...,...,...,...,...,...,...
302,2010-10-27,68,65,6,100,16.766563
303,2010-10-28,51,49,1,100,16.766563
304,2010-10-29,42,37,2,80,16.766563
305,2010-10-30,34,29,3,40,16.766563


In [41]:
df_final.to_csv("Clean_6h_Forecast.csv", index=False)